# Problem 1

### Initialization

In [1]:
# Formatting option for presentable plots:
%config InlineBackend.figure_format = 'svg' 

# Formatting option to display results
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# External libraries; see Appendix
from external import * 

# Self-written functions; see Appendix
from internal import *

#### Basic Random Variables

In [2]:
names  = np.array( ['EI',            'K',        'P'])
stddev = np.array([[1236.1606, 0.1/np.sqrt(3),  1000]]).T
Mx     = np.array([[8194.7583,            0.6,  5000]]).T
Rxx    = np.eye(3)
Dx = np.array([[stddev[j,0]*(j==i) for j in range(3)] for i in range (3)]);

##### Marginal distributions

In [3]:
i = 0
s = np.sqrt(np.log(Dx[i,i]**2/Mx[i,0]**2+1))
mu = Mx[i,0]**2/np.sqrt(Dx[i,i]**2+Mx[i,0]**2)
X0 = stats.lognorm(s=s, scale=mu);

In [4]:
i = 1 
a = 0.5
b = 0.7
X1 = stats.uniform(loc=a, scale=(0.7-0.5))
X1.cdf = lambda x: (x-0.5)/0.2
X1.pdf = lambda x: 1/0.2
X1.ppf = lambda x: x*(0.2) + 0.5

In [5]:
i = 2
X2 = stats.norm(loc=Mx[i,0], scale=Dx[i,i]);

#### Events

In [6]:
def g_x1(x):
    return np.pi**2*x[0]/(x[1]*4)**2-x[2]
# grad_x1 = grad(g_x1) 

def grad_x1(x): return np.array([np.pi**2/(x[1]*4)**2,
                                 -2*(x[1]*4)**(-3)*4*np.pi**2*x[0],
                                 -1.0])


## Transformation

The Nataf is not needed as the variables are independent, but the self-written class `Nataf` is used because it contains useful methods for displaying results.

Transformation:
$$\begin{aligned}
&U_{0}=\frac{\ln x_{0}-\lambda}{\zeta}\\
&U_{1}=\Phi^{-1}\left[\frac{x_{1}-a}{b-a}\right]\\
&U_{2}=\frac{x_{2}-\mu}{\sigma}
\end{aligned}$$

Jacobian:

$$\begin{array}{l}
\frac{\partial u_{0}}{\partial x_{0}}=\frac{1}{\zeta x_{0}} \\
\frac{\partial u_{1}}{\partial x_{1}}=\frac{1}{\phi\left(u_{1}\right)} \frac{1}{b-a} \\
\frac{\partial u_{2}}{\partial x_{2}}=\frac{1}{\sigma}
\end{array}$$

In [7]:
C = np.empty((3,3), dtype=object)

# X = solrel.siSet([X0,X1,X2]); # Statistically independent random variable set
X = solrel.nataf(rvset=[X0,X1,X2], Rxx=Rxx, C=C,names=names);
X.check_moments()

vars: ['X1' 'X2' 'X3']
mean: [8.194758e+03 6.000000e-01 5.000000e+03]
stdv: [1.236161e+03 5.800000e-02 1.000000e+03]
L0 : 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Rzz: 
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Jxu: 
 [[1215.    0.    0.]
 [   0.    0.    0.]
 [   0.    0. 1000.]]


## iHL-RF Step

In [8]:
x0 = np.array([6000, 0.6, 7000])
u0 = X.x_to_u(x0)

array([-2.00323498e+00, -2.78291642e-16,  2.00000000e+00])

In [9]:
func_u = lambda u:    g_x1(X.u_to_x(u))
grad_u = lambda u: grad_x1(X.u_to_x(u)).T @ X.Jxu(u)

ihlrf = iHLRF(func_u, grad_u, u0=u0)
ihlrf.init(verbose=True)
ihlrf.step(verbose=True);
# ihlrf.init()
# ihlrf.run()


iHL-RF Algorithm (Zhang and Der Kiureghian 1995)*************** 
Initialize iHL-RF: 
 u0:  [-2.0032 -0.      2.    ] 
 G0:  3280.8379 
 ui:  [[-2.0032 -0.      2.    ]] 
 Gi:  3280.8379 
 GradGi:  [ 1542.1257 -2734.3073 -1000.    ] 
 alphai:  [[-0.4681  0.8299  0.3035]] 
 spec1:  1.0 
 spec2:  2.3721 


iHL-RF step: 1
ui:  
 [[-1.1891]
 [ 2.1085]
 [ 0.7711]] 
 Gi:  2849.1588 
 GradGi:  [ 1293.0403  -213.908  -1000.    ] 
 alphai:  [[-0.7844  0.1298  0.6066]] 
 spec1:  0.8684 
 spec2:  1.911 



## FORM Results

### Form using previously written function

In [10]:
form = solrel.FORM(X,g_x1, grad_x1).run()
form.print_analysis()



FORM Results:
u*:    [-2.524  1.424  2.304]
x*:    [5.548783e+03 6.850000e-01 7.304124e+03]
alpha: [-0.6818  0.3845  0.6223]
beta:  3.7026
pf1:   0.000107


### Manual Computation of FORM Variables

In [11]:
x_star = np.array([5549.0, 0.6845, 7304.0]) # given
u_star = X.x_to_u(x_star)

array([-2.5241799 ,  1.42209043,  2.304     ])

In [12]:
alpha = -grad_u(u_star)/linalg.norm(grad_u(u_star))

array([-0.68160651,  0.38538363,  0.62200645])

In [13]:
beta = alpha @ u_star[:,None]

array([3.70165068])

In [14]:
pf1 = Phi(-beta)

array([0.0001071])

## Importance & Nature of Variables

Because the random variables are statistically independent, $\alpha$ (`alpha`) may be interpreted as an importance measure for the original (untransformed) variables.

$$\operatorname{Imp}(EI) > \operatorname{Imp}(P) > \operatorname{Imp}(K)$$

The nature of the basic random variables is determined from the sign of $\alpha$, and these align with the intuitive interpretation for the problem:
- $EI$: **Capacity**
- $K$: **Demand**
- $P$: **Demand**

## Sensitivities

$$J_{u,(\mu,\sigma)}=\left[\begin{array}{cc}
0 & 0 \\
0 & 0 \\
\frac{-1}{\sigma} & \frac{-x_{2}}{\sigma^{2}}
\end{array}\right]$$

In [15]:
J_u_musig = np.zeros((3,2))
J_u_musig[2,0] = -1/Dx[2,2]
J_u_musig[2,1] = -x_star[2]/Dx[2,2]**2
J_u_musig

array([[ 0.      ,  0.      ],
       [ 0.      ,  0.      ],
       [-0.001   , -0.007304]])

In [16]:
alpha @ J_u_musig

array([-0.00062201, -0.00454314])

## Approx. Effect of $L$

In [17]:
grad_gx_L = lambda x: -2*(x[1]*4)**(-3)*x[1]*np.pi**2*x[0]
dbeta_dL = grad_gx_L(x_star)/linalg.norm(grad_u(u_star))

-2.2720216928641213

In [18]:
new_beta = beta - 0.1*dbeta_dL

array([3.92885285])

## Updated Reliability

In [19]:
alpha2 = np.array([[ 0.885, -0.466]])
u_star2 = np.array([-2.783, 1.469])
beta2 = (alpha2 @ u_star2[:,None])[0]

array([-3.147509])

In [20]:
Pr_e2 = Phi(-beta2)[0]

0.9991766598168397

In [21]:
B = np.array([-beta,-beta2])[:,0]

array([-3.70165068,  3.147509  ])

In [22]:
Pr_e1e2 = MVN(B, cov=np.eye(2))

0.00010701251288513536

In [23]:
Pr_updated = Pr_e1e2/Pr_e2

0.00010710069318948258

In [24]:
beta3 = -invPhi(Pr_updated)

3.7016506827903553

## Lifetime Reliability

A rough approximation for one failure in 4 statistically independent events is given by the binomial distribution as follows:

In [25]:
stats.binom.pmf(n=4,k=1,p=pf1)

array([0.00042827])

TIME

# Appendix

## External imports

In [26]:
import itertools

# Scientific computing libraries
import numpy as np # Array manipulation / matrix computations
import scipy.linalg as linalg # Additional linear algebra functions
import scipy.stats as stats # Probability distributions
import scipy.optimize as optimize 
from autograd import grad # Algorithmic differentiation

import matplotlib.pyplot as plt # Plotting library
from mpl_toolkits.mplot3d import Axes3D

# Convenient references to standard normal functions
phi    = stats.norm.pdf
Phi    = stats.norm.cdf
invPhi = stats.norm.ppf
mvn = stats.multivariate_normal.pdf
MVN = stats.multivariate_normal.cdf

<bound method multivariate_normal_gen.cdf of <scipy.stats._multivariate.multivariate_normal_gen object at 0x0000015FBFED4608>>

## Internal imports

In [27]:
# My functions
import solrel
import solrel.systems as systems
from solrel.systems import Event
from ema.utilities.numerical import iHLRF

## My Code

### `class siSet`

In [28]:
class siSet(np.ndarray):
    """Statistically independent random variables"""
    def __new__(cls, rvset):
        return np.asarray(rvset).view(cls)

    def __init__(self, rvset):
        self.rvset = rvset
        n_dist = len(rvset)
        self.ndist = n_dist

        #  check if all distributions have finite moments
        for x in self.rvset:
            if (not((np.isfinite(x.mean()) and
                     np.isfinite(x.std())))):
                raise RuntimeError("The marginal distributions need to have "
                                   "finite mean and variance")


    def u_to_x(self,u):
        return np.array([X.ppf(stats.norm.cdf(u[i])) for i,X in enumerate(self.rvset)])
    
    def x_to_u(self,x):
        return np.array([stats.norm.ppf(X.cdf(x[i])) for i,X in enumerate(self.rvset)])
    
    def Jxu(self,u):
        X = self.rvset
        snorm = stats.norm.pdf
        d = np.identity(self.ndist)
        x = self.u_to_x(u)
        # return np.array ([[X[i].pdf(x[i])/snorm(u[i])*d[i,j] for i in range(self.ndist)] for j in range(self.ndist)])
        return np.array ([[snorm(u[i])/X[i].pdf(x[i])*(i==j) for i in range(self.ndist)] for j in range(self.ndist)])
    
    def Jux(self,x):
        u = self.x_to_u(x)
        Jux = np.linalg.inv(self.Jxu(u))
        return Jux

    # def Jux(self,x):
    #     z = self.x_to_z(x)
    #     u = self.x_to_u(x)
    #     snorm = stats.norm.pdf
    #     J = np.array ([[1/snorm(u[i])*(i==j)*X[i].pdf(x[i]) for i in range(self.ndist)] for j in range(self.ndist)])
    #     return self.invL0@J

    # def Jxu(self,u):
    #     return np.linalg.inv(self.Jux(self.u_to_x(u)))

    def jpdf(self,x,y):
        return 


### `class nataf`

In [29]:
  class nataf(np.ndarray):
    """Nataf transformation (Liu and Der Kiureghian 1986)"""
    def __new__(cls, rvset, Rxx, C,names=None):
        return np.asarray(rvset).view(cls)

    def __init__(self, rvset, Rxx, C,names=None):
        self.rvset = rvset
        self.Rxx = Rxx
        self.C = C
        n_dist = len(rvset)
        self.ndist = n_dist

        self.Rzz = np.identity(n=n_dist)
        if self.ndist == 2:
            rho = self.Rxx[0,1]
            deli = self.rvset[0].std()/self.rvset[0].mean()
            delj = self.rvset[1].std()/self.rvset[1].mean()
            self.Rzz[0,1] = self.Rzz[1,0] = rho*self.C(deli,delj,rho)
        else:
            for i in range(self.ndist):
                for j in range(i+1,self.ndist):
                    if C[i,j] is not None:
                        rho = self.Rxx[i,j]
                        deli = self.rvset[i].std()/self.rvset[i].mean()
                        delj = self.rvset[j].std()/self.rvset[j].mean()
                        self.Rzz[i,j] = self.Rzz[j,i] = rho*self.C[i,j](deli,delj,rho)

        self.L0 = np.linalg.cholesky(self.Rzz)
        self.invL0 = np.linalg.inv(self.L0)

    def check_moments(self):
        dists = np.array(['X'+str(1+i) for i in range(self.ndist)])
        means = np.around([X.mean() for X in self.rvset],3)
        stdvs = np.around([X.std() for X in self.rvset],3)
        print('vars:',dists)
        print('mean:',means)
        print('stdv:',stdvs)

        print('L0 : \n',np.around(self.L0,3))
        print('Rzz: \n',np.around(self.Rzz,3))
        print('Jxu: \n',np.around(self.Jxu(np.zeros(self.ndist)),0))
        return

    def x_to_u(self,x):
        return self.z_to_u(self.x_to_z(x))

    def u_to_x(self,u):
        return self.z_to_x(self.u_to_z(u))

    def x_to_z(self,x):
        return np.array([stats.norm.ppf(X.cdf(x[i])) for i,X in enumerate(self.rvset)])
    
    def z_to_x(self,z):
        return np.array([X.ppf(stats.norm.cdf(z[i])) for i,X in enumerate(self.rvset)])
    
    def z_to_u(self,z):
        return self.invL0@z

    def u_to_z(self,u):
        return self.L0@u

    def Jxu(self,u):
        z = self.u_to_z(u)
        x = self.u_to_x(u)
        X = self.rvset
        snorm = stats.norm.pdf
        d = np.identity(self.ndist)
        J = np.array ([[snorm(z[i])*d[i,j]/X[i].pdf(x[i]) for i in range(self.ndist)] for j in range(self.ndist)])
        return J@self.L0
    
    def Jux(self,x):
        return np.linalg.inv(self.Jxu(self.x_to_u(x)))

    def jpdf(self,x,y):
        X = self.rvset
        zx,zy = self.x_to_z([x,y])
        
        pos = np.empty(zx.shape + (2,))
        pos[:, :, 0] = zx
        pos[:, :, 1] = zy
        
        mvn = stats.multivariate_normal.pdf(pos, np.zeros(2),cov=self.Rzz)
        return mvn*(X[0].pdf(x)/stats.norm.pdf(zx)) * X[1].pdf(y)/stats.norm.pdf(zy)
    

### `class FORM`

In [30]:
class FORM:
    """First order reliability analysis"""
    def __init__(self, rvset, func_x, grad_x, sensitivity=False):
        self._hasrun = False
        self.rvset = rvset
        self.func_x = func_x
        self.grad_x = grad_x
        X = rvset
        self.func_u = lambda u: func_x(X.u_to_x(u))
        self.grad_u = lambda u: grad_x(X.u_to_x(u)).T @ X.Jxu(u)
        self.sensitivity = sensitivity

    
    def run(self, optimizer='scipy',loss='norm',verbose=False):
        G = self.func_u
        grad_u = self.grad_u
        ndim = len(self.rvset.rvset)
        u0 = self.rvset.x_to_u(np.array([x.mean() for x in self.rvset]))
        
        # loss
        def f(u): return scipy.linalg.norm(u)

        if optimizer=='ihlrf':
            u = self.design_point_u = iHLRF(G,grad_u,u0,loss=scipy.linalg.norm).run(verbose=verbose)[:,0].T
        else:
            options = {'disp': verbose}
            con = scipy.optimize.NonlinearConstraint(G, 0., 0.)
            sol = minimize(f,np.zeros(ndim),constraints=con,options=options)
            self.sol = sol
            if verbose: print(sol)
            u = self.design_point_u = sol.x

        alpha = -grad_u(u)/scipy.linalg.norm(grad_u(u))
        self.alpha = alpha
        self.design_point_x = self.rvset.u_to_x(u)
        self.beta = alpha@u
        self.pf = scipy.stats.norm.cdf(-self.beta)

        # Sensitivities
        if self.sensitivity:
            rvset=self.rvset
            Jxu = rvset.Jxu
            self.alpha = self.alpha
            u = self.design_point_u 
            x = self.design_point_x
            self.Mxl = x - Jxu(u)@u
            self.Sxlxl = Jxu(u)@Jxu(u).T
            self.Dxl = np.array([[np.sqrt(self.Sxlxl[0,0]),0],[0.0, np.sqrt(self.Sxlxl[1,1])]])
            self.gamma = self.alpha@rvset.Jux(x)@self.Dxl/np.linalg.norm(self.alpha@rvset.Jux(x)@self.Dxl)
        
        self._hasrun = True
        return self

    def print_analysis(self,form=True,sensitivity=False):
        if not self._hasrun: self.run()

        if form:
            print('\n')
            print('FORM Results:')
            print('u*:    {}'.format(np.around(self.design_point_u,3)))
            print('x*:    {}'.format(np.around(self.design_point_x,3)))
            print('alpha: {}'.format(np.around(self.alpha,4)))
            print('beta:  {}'.format(np.around(self.beta,4)))
            print('pf1:   {}'.format(np.around(self.pf,6)))

        if sensitivity:
            if self.gamma is None: self.run(sensitivity=True)
            print('\n')
            print('Sensitivity:')
            print('alpha: {}'.format(np.around(self.alpha,4)))
            print('gamma: {}'.format(np.around(self.gamma,4)))


### `class iHLRF`

In [31]:
class iHLRF:
    def __init__(self, f, gradf, u0, loss=None, tol1=1.0e-4, tol2=1.0e-4, maxiter=20, maxiter_step=20):
        self.u0 = u0
        self.f = f 
        self.gradf = gradf
        self.tol1 = tol1
        self.tol2 = tol2
        self.maxiter = maxiter
        self.maxiter_step = maxiter_step
        self.loss = loss
        if loss is None: 
            self.loss = np.linalg.norm

        # self.init()

    def init(self,verbose=False):
        self.G0 = self.f(self.u0) # scale parameter
        self.ui = self.u0[None,:].T
        self.Gi = self.f(self.ui[:,0].T)
        self.GradGi = self.gradf(self.ui[:,0].T)
        self.alphai = -(self.GradGi / np.linalg.norm(self.GradGi))[None,:]
        self.count = 0
        self.spec1 = abs(self.Gi/self.G0)
        self.spec2 = self.loss(self.ui - (self.alphai@self.ui)*self.alphai.T )

        if verbose:
            print('\niHL-RF Algorithm (Zhang and Der Kiureghian 1995)***************',
                  '\nInitialize iHL-RF: \n',
                  'u0: ', np.around(self.u0.T,4), '\n',
                  'G0: ',np.around(self.G0,4),'\n',
                  'ui: ', np.around(self.ui.T,4), '\n',
                  'Gi: ', np.around(self.Gi,4), '\n',
                  'GradGi: ', np.around(self.GradGi,4),'\n',
                  'alphai: ', np.around(self.alphai,4),'\n',
                  'spec1: ' , np.around(self.spec1,4),'\n',
                  'spec2: ' , np.around(self.spec2,4),'\n',)
    
    def incr(self,basic=False, verbose=False):
        if basic == True:
            self.ui1 = self.ui + self.lamda * self.di
            return self.ui1 

        self.ci = self.loss(self.ui) / np.linalg.norm(self.GradGi) + 10.0
        self.mi = 0.5*self.loss(self.ui)**2 + self.ci*abs(self.Gi)
        self.mi1 = 0.5*self.loss(self.ui1)**2 + self.ci*abs(self.Gi1)

        self.count_step = 0
        while (self.mi1 >= self.mi) and (self.count_step <= self.maxiter_step):
            self.lamda = self.lamda/2
            self.ui1 = self.ui + self.lamda * self.di
            self.Gi1 = self.f(self.ui1[:,0].T)
            self.mi1 = 0.5*np.linalg.norm(self.ui1)**2 + self.ci*abs(self.Gi1)
            self.count_step += 1

        return self.ui1
    
    def dirn(self):
        self.di = (self.Gi/np.linalg.norm(self.GradGi) + self.alphai@self.ui)*self.alphai.T - self.ui
        return self.di

    def step(self,verbose=False,basic=False):
        # self.di = (self.Gi/np.linalg.norm(self.GradGi) + self.alphai@self.ui)*self.alphai.T - self.ui
        di = self.dirn()
        self.lamda = 1.0
        self.ui1 = self.ui + self.lamda * di
        self.Gi1 = self.f(self.ui1[:,0].T)

        ui1 = self.incr(basic)

        self.ui = ui1  
        self.Gi = self.f(self.ui[:,0].T)
        self.GradGi = self.gradf(self.ui[:,0].T)
        self.alphai = -(self.GradGi / np.linalg.norm(self.GradGi))[None,:]

        self.spec1 = abs(self.Gi/self.G0)
        self.spec2 = self.loss(self.ui - (self.alphai@self.ui)*self.alphai.T)
        self.count += 1


        if verbose: print('\niHL-RF step: {}'.format(self.count))

        if verbose:
            print('ui: ', '\n', np.around(self.ui,4), '\n',
                  'Gi: ',       np.around(self.Gi,4), '\n',
                  'GradGi: ', np.around(self.GradGi,4),'\n',
                  'alphai: ', np.around(self.alphai,4),'\n',
                  'spec1: ' , np.around(self.spec1,4),'\n',
                  'spec2: ' , np.around(self.spec2,4),'\n',)
        return self.ui
    
    def run(self,verbose=False, steps=None):
        self.init(verbose=verbose)
        if steps is not None: self.maxiter = steps
        while not(self.spec1 < self.tol1 and self.spec2 < self.tol2):
            self.step(verbose=verbose)

            if (self.count > self.maxiter): break

        return self.ui